In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from hyperopt import hp, fmin, tpe, Trials
import itertools

In [2]:
train = '/mnt/NewVolume/DUMP/Notebook/xray_analytics_cnn/data/BoneFractureDataset/training'
test = '/mnt/NewVolume/DUMP/Notebook/xray_analytics_cnn/data/BoneFractureDataset/testing'

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
#PreProcessing

In [4]:
def scale_images(x, y):
    return x / 255, y

In [5]:
train_data = tf.keras.utils.image_dataset_from_directory(train, image_size=(256, 256), labels="inferred", label_mode="binary", validation_split=0.2, subset="training",seed=42)
train_data = train_data.map(scale_images)
train_size = int(len(train_data))
train_data_final = train_data.take(train_size)
print(train_size)

Found 8863 files belonging to 2 classes.
Using 7091 files for training.
222


In [6]:
val_data = tf.keras.utils.image_dataset_from_directory(train, image_size=(256, 256), labels="inferred", label_mode="binary", validation_split=0.2, subset="validation", seed=42)
val_data = val_data.map(scale_images)
val_data_final = val_data.take(len(val_data))
print(len(val_data))

Found 8863 files belonging to 2 classes.
Using 1772 files for validation.
56


In [7]:
#AlexNet

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from hyperopt import hp, fmin, tpe, Trials

# Define the search space for hyperparameters
space = {
    'learning_rate': hp.choice('learning_rate', [1e-3, 1e-4, 1e-5, 1e-6]),  # Learning rate choice
    'optimizer': hp.choice('optimizer', [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop]),  # Optimizer choice
    'neurons': hp.choice('neurons', [64, 128, 256]),  # Number of neurons in the activation layer
    'activation_function': hp.choice('activation_function', ['relu', 'tanh', 'sigmoid'])  # Activation function choice
}

# Define the objective function to optimize
def objective(params):
    model = keras.models.Sequential([
        keras.layers.Conv2D(filters=128, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(256, 256, 3)),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=(3, 3)),
        keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPool2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(params['neurons'], activation=params['activation_function']),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(params['neurons'], activation=params['activation_function']),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid')  # Change the number of neurons and activation function
    ])

    # Create an optimizer based on the choice in params
    optimizer = params['optimizer'](learning_rate=params['learning_rate'])

    # Compile the model with the chosen optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model with early stopping (replace train_data and val_data_final with your data)
    history = model.fit(train_data_final, epochs=10, validation_data=val_data_final, verbose=0, callbacks=[early_stopping])

    # Calculate the validation accuracy
    val_accuracy = np.max(history.history['val_accuracy'])
    
    # Return the negative accuracy (to be minimized)
    return -val_accuracy

# Create a Trials object to keep track of the optimization process
trials = Trials()

# Perform Bayesian optimization to find the best hyperparameters
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

# Print the best hyperparameters found
print("Best hyperparameters:")
print(best)

100%|███████| 20/20 [35:02<00:00, 105.14s/trial, best loss: -0.9915350079536438]
Best hyperparameters:
{'activation_function': 0, 'learning_rate': 1, 'neurons': 0, 'optimizer': 2}
